# Run Sequence 


`IDT_Ordering_Module`
- Removes duplicate oligos and creates IDT ordering format. <br>

`Module 1` 
- Re-formats the IDT information sheet.  <br>

`Module 2` 
- Add water to primers/guides, create picklists for Fluent.
- Create picklist for transfer to 384well Echo source plate for PCR setup.

`Module 3` 
- Picklist for guide mixing and phosphorylation. <br>

`Module 4`
- Picklist for primer and template mixing in Echo.
- There is an extension module for module 4 meant to be used for extension PCR. 

`Module 5`
- Equimolar mixing of PCR fragments.

Each module is dependent on successful run of the previous module. 

# Module 2

### Add water to Primers and Guides 


Read the file with names starting in following format.<br> 
> `IDT_Primers_` <br>
> `IDT_Guides_`<br>

Add water to dilute primers using TECAN Fluent and .gwl format worklist. 

### Worklist for transfer to Corning 384-well at 10x dilution for ECHO source plate

Transfer the primers from 96-well/384-well plate to corning 384-well plate using Fluent. 
Dilution : 10x
total volume: 60ul 

    6ul + 54 ul

This plate will be used for setting up PCR on ECHO. 

This script is same for both IDT order received in 96-well or 384-well plasmid. It requires the column name to be`384_well_position` in the IDT_Primers file. 

In [3]:
import os, sys
import traceback
import pandas as pd
import re
import math
import numpy as np
from string import ascii_uppercase
import glob
import shutil


print(os.getcwd())

def All_Plate_types():
    #global Primer_plate_type
    #Primer_plate_type = ''
    global Plate_types 
    Plate_types = dict()
    

def write_to_subfolder(filename, dataframe_object, subdirectory):
    
    curr_dir = os.getcwd()
    new_dir = os.path.join(curr_dir,subdirectory)
    counter = 0
    
    if curr_dir.find(subdirectory) == -1:
        
        try:
            os.mkdir(subdirectory)
        except Exception:
            pass
    
        os.chdir(new_dir)
        print("File written :-   ", filename)
        dataframe_object.to_csv(filename, index=False)
        counter = 1
        os.chdir('..')
        
    else:
        os.chdir(new_dir)
        print("File written :-   ", filename)
        dataframe_object.to_csv(filename, index=False)
        os.chdir('..')

def Read_Primer_Guide_Files ():
    
    All_Plate_types()
    
    Primers = glob.glob("**/*IDT_Primers_*")
    Guides = glob.glob("**/*IDT_Guides_*")

    if Primers and Guides:
        Dilution_picklist(Primers[0])
        Dilution_picklist(Guides[0])
        add_384well_position(Primers[0])

    else:
        print('Verify if IDT_Primers_ and IDT_Guides_ files exists in subfolder!!!')

                
def Dilution_picklist (filename):
        
    print('file read:- ', filename)
    df = pd.read_csv(filename)  
                
    column_names = ["Source Well", "Destination Well", "Volume"]
    df_picklist = pd.DataFrame(columns = column_names)
    
    
    Well_list =  df['Well Position'].tolist()
    if "A13" in Well_list:
        Multiplier = 5
    else:
        Multiplier = 10
        
    if 'Guide' in filename:
        Plate_types['Guide_dilution'] =  [Multiplier]
    elif 'Primer' in filename:
        Plate_types['Primer_dilution'] =  [Multiplier]
    
    for index, row in df.iterrows():
        name = df['Sequence Name'].loc[index]
        nmol = df['nmoles'].loc[index]
        temp = re.findall('[0-9][0-9].[0-9]', str(nmol))
        if not temp:
            temp = re.findall('[0-9][0-9]', nmol)
            
        temp = math.ceil(float(temp[0]))
        h20_vol = int((temp * Multiplier))
        Source = 'A1'
        
        Dest = df['Well Position'].loc[index]
                  
        list1 = [Source, Dest, h20_vol]
        a_series = pd.Series(list1, index = df_picklist.columns)
        df_picklist = df_picklist.append(a_series, ignore_index=True)
        
    filename = filename.split('/')[1]
    new_file  = '_hydration_'+filename[4:-4]+'.csv'    
    Fluent_picklist(new_file, df_picklist, 'Dilution_Picklist')
    
    
def IDT_plate_definition(Filename, df_picklist):
    
    Well_list = df_picklist['Destination Well'].tolist()
    global Primer_plate_type

    if 'Primers' in Filename:
        Filename = 'Worklist_1'+Filename
        
        if "A13" in Well_list:
            Dest_plate = 'Primers384well'
        else:
            Dest_plate = 'Primers96Deepwell'
       
        Primer_plate_type = Dest_plate
        Plate_types['Primer_plate'] =  [Dest_plate]
        
    elif 'Guides' in Filename:        
        Filename = 'Worklist_4'+Filename

        if "A13" in Well_list:
            Dest_plate = 'Guides384well'
        else:
            Dest_plate = 'Guides96Deepwell'
        
        Plate_types['Guide_plate'] = [Dest_plate]
    
    return Dest_plate, Filename

    
def Fluent_picklist(Filename, df_picklist, subfolder):
    
    df_picklist['Source_plate'] = '300mlSBSWater'
    # the names of Source_plate and Dest_plate will change with different plate types
    Dest_plate, Filename = IDT_plate_definition(Filename, df_picklist)

    df_picklist['Dest_plate'] = Dest_plate
        
    cols = ['Source_plate', 'Source Well', 'Dest_plate', 'Destination Well', 'Volume' ]
    df_picklist = df_picklist[cols]
    
    write_to_subfolder(Filename, df_picklist, subfolder)
    CSV_to_GWL_add_water(Filename, df_picklist, 'F;\n')
    

def CSV_to_GWL_add_water(Filename, Fluent_picklist, transfer_mode):
    
    Step_list =[]

    for index, row in Fluent_picklist.iterrows():

        Volume = Fluent_picklist['Volume'].loc[index]

        S_plate = Fluent_picklist['Source_plate'].loc[index]
        S_well = Fluent_picklist['Source Well'].loc[index]
        Step_1 = str('A;'+ S_plate+';;;'+str(S_well)+';;'+str(Volume)+';;;;\n')
        Step_list.append(Step_1)

        D_plate = Fluent_picklist['Dest_plate'].loc[index]
        D_well = Fluent_picklist['Destination Well'].loc[index]
        Step_2 = str('D;'+ D_plate+';;;'+str(D_well)+';;'+str(Volume)+';;;;\n')
        Step_list.append(Step_2)
        
        Step_list.append(transfer_mode)

    Gwl_Filename = Filename[:-4]+'.gwl'
    
    F = open(Gwl_Filename, "w+")
    F.writelines(Step_list)
    F.close()
    
    
def add_384well_position(Primer_filename):
    
    df_IDT_primers = pd.read_csv(Primer_filename)
    
    df_384well = []
    for letter in ascii_uppercase:
        for i in range(24):
            Name = letter+str(i+1)
            df_384well.append(Name)      

            if letter == 'P':
                break
    
    well_list = []
    counter = 0
    for index, row in df_IDT_primers.iterrows():
        well_list.append(df_384well[counter])
        counter = counter + 1
    
    df_IDT_primers['384_well_position'] = well_list
    write_to_subfolder(Primer_filename.split('/')[1], df_IDT_primers, 'Generated_Files')
    
    Primer_transfer_to_384well_forECHO_onOT2(df_IDT_primers)
    
    
def Primer_transfer_to_384well_forECHO_onOT2(df_primers):
    
    df_picklist_new = pd.DataFrame()
    list1 = []
    Vol = 5
    selected_columns = df_primers[["Well Position", "384_well_position"]]
    df_picklist_new = selected_columns.copy()
    df_picklist_new['Volume'] = 6
    df_picklist_new = df_picklist_new.rename(columns = {'Well Position':'Source Well' , 
                                                        '384_well_position':'Destination Well' })    
    
    Primer_filename_new ='Worklist_3_primers_to_384EchoSource.csv'
    write_to_subfolder(Primer_filename_new, df_picklist_new, 'Dilution_Picklist')
    
    df_picklist_new['Source_plate'] = Primer_plate_type
    df_picklist_new['Dest_plate'] = 'EchoSource384'
    
    cols = ['Source_plate', 'Source Well', 'Dest_plate', 'Destination Well', 'Volume' ]
    df_picklist_new = df_picklist_new[cols]
    df_picklist_new = df_picklist_new.reset_index(drop = True)
        
    CSV_to_GWL_add_water(Primer_filename_new, df_picklist_new, 'W;\n')
    df_picklist_new['Volume'] = 54
    df_picklist_new['Source_plate'] = '300mlSBSWater'
    df_picklist_new['Source Well'] = 'A1'
    
    cols = ['Source_plate', 'Source Well', 'Dest_plate', 'Destination Well', 'Volume' ]
    df_picklist_new = df_picklist_new[cols]
    df_picklist_new = df_picklist_new.reset_index(drop = True)
    
    Filename = 'Worklist_2_55ul_water_384EchoSource.csv'
    CSV_to_GWL_add_water(Filename, df_picklist_new, 'F;\n')
    
    Move_GWL_files_to_subfolder()

    
def Move_GWL_files_to_subfolder():
    
    source = os.getcwd()
    Dest_dir = os.path.join(source, 'gwl_worklists')
    print('\n Moving all gwl flies to:  \t', Dest_dir, '\n')

    if os.path.exists(Dest_dir):
        shutil.rmtree(Dest_dir)
    os.makedirs(Dest_dir)

    gwl_files = glob.glob('*.gwl')

    for file in gwl_files:
        file_name = os.path.join(source, file)
        shutil.move(file_name, Dest_dir)
        
    Plate_types['Echo Plate'] = ['EchoSource384']
    
    Plate_type_df = pd.DataFrame.from_dict(Plate_types, orient = 'index', columns = ['Value'])
    Plate_type_df = Plate_type_df.reset_index(drop = False)
    
    write_to_subfolder('Input_Plate_types.csv', Plate_type_df, 'gwl_worklists')
    print(Plate_type_df.to_string())
    
    
if __name__ == '__main__':
    
    Read_Primer_Guide_Files()
    


/Users/nilmani/Desktop/Python/High_GC_Picklist
file read:-  Generated_Files/IDT_Primers_highGC.csv
File written :-    Worklist_1_hydration_Primers_highGC.csv
file read:-  Generated_Files/IDT_Guides_highGC.csv
File written :-    Worklist_4_hydration_Guides_highGC.csv
File written :-    IDT_Primers_highGC.csv
File written :-    Worklist_3_primers_to_384EchoSource.csv

 Moving all gwl flies to:  	 /Users/nilmani/Desktop/Python/High_GC_Picklist/gwl_worklists 

File written :-    Input_Plate_types.csv
             index              Value
0  Primer_dilution                 10
1     Primer_plate  Primers96Deepwell
2   Guide_dilution                 10
3      Guide_plate   Guides96Deepwell
4       Echo Plate      EchoSource384


## END MODULE 2